<a href="https://colab.research.google.com/github/adityamengade/C-Codes/blob/master/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import keras
import PIL
import numpy as np
import os
from matplotlib.pyplot import imshow
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import regularizers
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau,EarlyStopping


from google.colab import drive
drive.mount('/content/drive/')
save_dir = os.path.join(os.getcwd(), 'cnn_model')
model_name = 'cnn_model.h5'
img_width, img_height = 128,128

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.050,
        height_shift_range=0.05,
        rescale=1/255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest')
train_data_dir = 'spiral/training'
validation_data_dir = 'spiral/testing'
nb_train_samples = 3000
nb_validation_samples = 1150
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_width, img_height)
else:
    input_shape = (img_width, img_height, 1)


model = Sequential()
model.add(Conv2D(64, (3,3),input_shape=(256,256, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

batch_size = 32


train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/spiral/training',
        color_mode='grayscale', 
        target_size=(256,256),  
        batch_size=batch_size,
        class_mode='binary')  

validation_generator = test_datagen.flow_from_directory(
        '/content/drive/My Drive/spiral/testing',
        color_mode='grayscale',
        target_size=(256,256),
        batch_size=batch_size,
        class_mode='binary')
reduce_lr_2 = ReduceLROnPlateau(monitor='val_loss',patience=6,rate=0.4,min_lr=1e-12,verbose=1)
early_stop_2 = EarlyStopping(monitor='val_loss',patience=12,verbose=1)
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=20,
        shuffle = True,
        validation_data=validation_generator,
        validation_steps=400 // batch_size,
        callbacks=[reduce_lr_2,early_stop_2],verbose=1)
model.save_weights('cnn_model.h5')


if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Found 832 images belonging to 2 classes.
Found 344 images belonging to 2 classes.
Epoch 1/20
62/62 [==============================] - 14s 223ms/step - loss: 0.6694 - acc: 0.5706 - val_loss: 0.6616 - val_acc: 0.5346
Epoch 2/20
62/62 [==============================] - 13s 209ms/step - loss: 0.5934 - acc: 0.6522 - val_loss: 0.6490 - val_acc: 0.5452
Epoch 3/20
62/62 [==============================] - 13s 214ms/step - loss: 0.5561 - acc: 0.6689 - val_loss: 0.8729 - val_acc: 0.5160
Epoch 4/20
62/62 [==============================] - 13s 217ms/step - loss: 0.5624 - acc: 0.6759 - val_loss: 0.7077 - val_acc: 0.5452
Epoch 5/20
62/62 [==============================] - 13s 211ms/step - loss: 0.5445 - acc: 0.6724 - val_loss: 0.6967 - val_acc: 0.5984
Epoch 6/20
62/62 [==============================] - 14s 218ms/step - loss: 0.5435 - acc: 0.6719 - val_loss: 0.9133 - val_a

In [2]:
import cv2
from keras.preprocessing import image
import numpy as np
from keras.applications.imagenet_utils import preprocess_input
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)
img= image.load_img('/content/drive/My Drive/h.png', target_size=(150,150))
print(img)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print("result")
res=model.predict(x)
print(res)

Mounted at /content/drive/
<PIL.Image.Image image mode=RGB size=150x150 at 0x7F60BA1E8748>
result


ValueError: ignored

In [0]:
import tensorflow as tf

model.save('cnn_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model_file('cnn_model.h5')
tflite_model = converter.convert()
open('cnn_model.tflite', 'wb').write(tflite_model)

try:
    from google.colab import files
    files.download('cnn_model.tflite')
except:
    print("Skip downloading")

In [0]:

from keras.utils import plot_model
plot_model(model,to_file="a.png")
